In [1]:
!pip install elasticsearch

In [2]:
!curl elasticsearch:9200

{
  "name" : "91afebcfa735",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "ilew_pR7RhqgqmPQv8pb2Q",
  "version" : {
    "number" : "8.5.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "4ed5ee9afac63de92ec98f404ccbed7d3ba9584e",
    "build_date" : "2022-12-05T18:22:22.226119656Z",
    "build_snapshot" : false,
    "lucene_version" : "9.4.2",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [3]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch(
    hosts=[{"scheme": "http", "host": "elasticsearch", "port": 9200}],
)

In [4]:
es_client.info()

ObjectApiResponse({'name': '91afebcfa735', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ilew_pR7RhqgqmPQv8pb2Q', 'version': {'number': '8.5.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '4ed5ee9afac63de92ec98f404ccbed7d3ba9584e', 'build_date': '2022-12-05T18:22:22.226119656Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Getting movielens-100k dataset

In [13]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ./ml-100k.zip
!rm ml-100k.zip

In [18]:
import pandas as pd

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,
                    encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)
lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076


In [21]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [22]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [23]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [20]:
# ratings matrix
rating_matrix = pd.pivot_table(lens, index='user_id', columns=['movie_id'], values=['rating'])
rating_matrix.head()

rating                                               ...            \
movie_id   1    2    3    4    5    6    7    8    9    10    ... 1673 1674   
user_id                                                       ...             
1           5.0  3.0  4.0  3.0  3.0  5.0  4.0  1.0  5.0  3.0  ...  NaN  NaN   
2           4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  ...  NaN  NaN   
3           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
4           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
5           4.0  3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

                                                  
movie_id 1675 1676 1677 1678 1679 1680 1681 1682  
user_id                                           
1         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
5         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 1682 columns]

## Compute embeddings through Matrix Factorization

In [43]:
from scipy.sparse.linalg import svds
import numpy as np

EMBEDDING_DIM = 10
U, sigma, Vt = svds(rating_matrix.fillna(0).values, k=EMBEDDING_DIM)

In [44]:
sigma

array([ 99.74793974, 106.8291837 , 121.90769976, 126.57977314,
       145.87261327, 158.21191449, 159.15359872, 217.84622472,
       244.83634567, 640.63362257])

In [45]:
U.shape, Vt.shape

((943, 10), (10, 1682))

In [46]:
user_embeddings = U
movie_embeddings = Vt.transpose()

user_embeddings.shape, movie_embeddings.shape

((943, 10), (1682, 10))

In [47]:
np.dot(U, sigma).shape

(943,)

In [54]:
np.matmul(U, Vt)

array([[ 8.26712082e-03,  7.09071179e-03,  3.48595178e-03, ...,
        -5.78515697e-05,  7.74297711e-05,  3.46825894e-04],
       [ 5.29302775e-03, -1.54390957e-04, -5.66437190e-04, ...,
         4.02232229e-05,  1.64155393e-05,  1.48454733e-05],
       [-3.00079924e-03, -7.09702110e-04,  2.02858438e-04, ...,
         1.16499323e-04, -3.24804504e-05, -7.92476502e-05],
       ...,
       [ 1.02001429e-02, -9.97150943e-04,  1.10522946e-03, ...,
        -1.17501320e-05, -3.45286123e-05, -3.96582111e-05],
       [ 2.54168180e-03, -1.28716876e-03, -3.66837925e-03, ...,
         8.85722586e-05, -1.43620366e-05, -2.32173158e-04],
       [-4.09924970e-03,  7.96698729e-03,  3.43940654e-03, ...,
        -5.74302943e-05,  1.35379251e-04,  1.32014771e-04]])

In [65]:
# rating_matrix_prediction = np.dot(np.matmul(U, sigma), Vt)

# Index data to Elasticsearch

## Users & Movies

In [104]:
users_index_name = 'users'
try:
    es_client.indices.delete(index=users_index_name)
except Exception as e:
    print(e)

In [105]:
users.head(1)

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711


In [106]:
settings = {
    "number_of_shards": 1,
    "number_of_replicas": 1
}
mappings = {
    'properties': {
        'user_id': {'type': 'long'},
        'age': {'type': 'long'},
        'sex': {'type': 'keyword'},
        'occupation': {'type': 'keyword'},
        'embedding': {'type': 'dense_vector', 
                      'dims': EMBEDDING_DIM,
                      'index': True,
                      # 'similarity': 'dot_product'
                      'similarity': 'cosine'
                     },
}}
request_body = {
    "settings" : settings,
    'mappings': mappings
}
es_client.indices.create(index=users_index_name, settings=settings, mappings=mappings)
# es_client.indices.create(index=index_name, body=request_body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'users'})

In [107]:
movies_index_name = 'movies'
try:
    es_client.indices.delete(index=movies_index_name)
except Exception as e:
    print(e)

In [108]:
movies.head(1)

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...


In [109]:
settings = {
    "number_of_shards": 1,
    "number_of_replicas": 1
}
mappings = {
    'properties': {
        'movie_id': {'type': 'long'},
        'title': {'type': 'text'},
        'embedding': {'type': 'dense_vector', 
                      'dims': EMBEDDING_DIM,
                      'index': True,
                      # 'similarity': 'dot_product'
                      'similarity': 'cosine'
                     },
}}
request_body = {
    "settings" : settings,
    'mappings': mappings
}
es_client.indices.create(index=movies_index_name, settings=settings, mappings=mappings)
# es_client.indices.create(index=index_name, body=request_body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'})

## Bulk Insert of Users

In [110]:
actions = []
for i, row in enumerate(users.iterrows()):
    action = {'index': {'_index': users_index_name, '_id': i}}
    doc = {
        'user_id': row[1].user_id,
        'sex': row[1].sex,        
        'age': row[1].age,
        'occupation': row[1].occupation,
        'embedding': list(user_embeddings[i,:])
    }
    actions.append(action)
    actions.append(doc)
es_client.bulk(index=users_index_name, operations=actions);

In [111]:
# check all docs have been inserted
result = es_client.count(index=users_index_name)
print(result.body['count'])

943


## Bulk Insert of Movies

In [112]:
actions = []
for i, row in enumerate(movies.iterrows()):
    action = {'index': {'_index': movies_index_name, '_id': i}}
    doc = {
        'movie_id': row[1].movie_id,
        'title': row[1].title,        
        'embedding': list(movie_embeddings[i,:])
    }
    actions.append(action)
    actions.append(doc)
es_client.bulk(index=movies_index_name, operations=actions);

ObjectApiResponse({'took': 126, 'errors': False, 'items': [{'index': {'_index': 'movies', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'movies', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'movies', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'movies', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'movies', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1, 'status': 201}}, {'index': {'_ind

In [113]:
# check all docs have been inserted
result = es_client.count(index=movies_index_name)
print(result.body['count'])

1682


## Query all

In [115]:
results = es_client.search(index=users_index_name, query={'match_all': {}})
print(len(results['hits']['hits']))
for result in results['hits']['hits']:
    print(result)

10
{'_index': 'users', '_id': '0', '_score': 1.0, '_source': {'user_id': 1, 'sex': 'M', 'age': 24, 'occupation': 'technician', 'embedding': [-0.01608566864629557, -0.05723586101854579, 0.016895296602515774, -0.08496671631774262, 0.017163603870490707, 0.014188048874729025, -0.08434724474949544, 0.006132555326231732, 0.0059750643916337874, -0.06580430564337676]}}
{'_index': 'users', '_id': '1', '_score': 1.0, '_source': {'user_id': 2, 'sex': 'F', 'age': 53, 'occupation': 'other', 'embedding': [0.004973217544389459, -0.036543318152696055, 0.005822618235664983, -0.017911066183493845, 0.005760289653048406, -0.01546748922065267, 0.016282469622077175, -0.0525785569653648, -0.04662602353058116, -0.014021043228925166]}}
{'_index': 'users', '_id': '2', '_score': 1.0, '_source': {'user_id': 3, 'sex': 'M', 'age': 23, 'occupation': 'writer', 'embedding': [0.007053754286173895, 0.022341430597562044, 0.0025147449042921513, -0.004541285268650615, 0.0038501128345949354, 0.044056880336302257, 0.02856415

## Personalised Recommendations

1. Select the user_id for which you want to show personalised recommendations
2. Get top movies recommended

In [119]:
## select the first user that shows up in the query


query = {'match_all': {}}
results = es_client.search(query=query, index=users_index_name)
print(len(results['hits']['hits']))
print(results['hits']['hits'][0]['_source'])
user_id = results['hits']['hits'][0]['_source']['user_id']
user_embedding = results['hits']['hits'][0]['_source']['embedding']
# for result in results['hits']['hits']:
#     print(result)

10
{'user_id': 1, 'sex': 'M', 'age': 24, 'occupation': 'technician', 'embedding': [-0.01608566864629557, -0.05723586101854579, 0.016895296602515774, -0.08496671631774262, 0.017163603870490707, 0.014188048874729025, -0.08434724474949544, 0.006132555326231732, 0.0059750643916337874, -0.06580430564337676]}


### Recommendations Without Filter

We should remove those that were already seen

In [124]:
%%time
query = {
    'script_score': {
        'query' : {'match_all': {}},
        'script': {
            "source": "dotProduct(params.queryVector, 'embedding') + 1",
            "params": {
              "queryVector": user_embedding
            }
    }
    },
    }
results = es_client.search(query=query, index=movies_index_name)
for result in results['hits']['hits']:
    print(result['_source']['title'])

Blade Runner (1982)
Much Ado About Nothing (1993)
Star Wars (1977)
Close Shave, A (1995)
Princess Bride, The (1987)
Brazil (1985)
Twelve Monkeys (1995)
Big Night (1996)
Fargo (1996)
Wrong Trousers, The (1993)
CPU times: user 2.03 ms, sys: 2.59 ms, total: 4.62 ms
Wall time: 7.39 ms


In [139]:
ratings.loc[ratings.user_id == user_id].merge(movies, on='movie_id')[['title', 'rating']].sort_values('rating', ascending=False).head(20)

,title,rating
136,Cinema Paradiso (1988),5
67,Brazil (1985),5
182,Maya Lin: A Strong Clear Vision (1994),5
186,Return of the Jedi (1983),5
191,Mystery Science Theater 3000: The Movie (1996),5
77,Dolores Claiborne (1994),5
75,"Postino, Il (1994)",5
74,"Usual Suspects, The (1995)",5
194,Sling Blade (1996),5
72,Kids in the Hall: Brain Candy (1996),5


# References on Elastic

* [Dense vectors](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html)

* [KNN search](https://www.elastic.co/guide/en/elasticsearch/reference/current/knn-search.html)

* [Vector functions](https://www.elastic.co/guide/en/elasticsearch/reference/master/query-dsl-script-score-query.html#vector-functions)


